Домашнее задание
Откуда берутся датасеты? Практический проект по сбору данных и работе с текстами 

Цель:
В этом домашнем задании вам предстоит обойти все ловушки серверов, пробраться сквозь страницы html-код, собрать себе свой собственный датасет и натренировать на нём модель. 

Описание/Пошаговая инструкция выполнения домашнего задания:
Часть 1. Парсинг

По аналогии с занятием, возьмите интересующий вас сайт, на котором можно пособирать какие-то данные (и при этом API не предоставляется). 

Идеальный датасет должен иметь текстовое описание некоторого объекта и некоторую целевую переменную, соответствующую этому объекту. Например:

Сайт новостей: текстовое описание - сама новость, целевая переменная - количество просмотров новости (можно поделить на число дней с момента даты публикации, чтобы получить “среднее число просмотров в день”)
Сайт с товарами/книгами/фильмами: текстовое описание товара/книги/фильма + средний рейтинг в качестве целевой переменной
Блоги - тексты заметок + число просмотров
И любые другие ваши идеи, которые подходят под такой формат
Напишите свой парсер, который будет бегать по страничкам и автоматически что-то собирать. 

Не забывайте, что парсинг - это ответственное мероприятие, поэтому не бомбардируйте несчастные сайты слишком частыми запросами (можно ограничить число запросов в секунду при помощи time.sleep(0.3), вставленного в теле цикла)

Часть 2. NLP 

Разбейте собранные данные на train/test, отложив 20-30% наблюдений для тестирования
Примените tf-idf преобразование для текстового описания. Используйте как отдельные токены, так и биграммы, отсейте стоп-слова, а также слова, которые встречаются слишком редко или слишком часто (параметры min/max_df), не забудьте убрать l2 регуляризацию, которая по умолчанию включена
Если в вашем датасете целевая переменная непрерывная (например, среднее число просмотров в день), то воспользуйтесь линейной регрессией, если дискретная (положительный/отрицательный отзыв), то логистической
Постройте регрессию с настройкой параметра регуляризации, оцените качество при помощи соответствующих задаче метрик
Визуализируйте получившиеся коэффициенты регрессии (возьмите топ-50 слов). Проинтерпретируйте результаты.
P.S. Если с парсингом не задалось или данных собралось слишком мало - не отчаивайтесь, главное, что ваш парсер собрал хоть что-то! А для второй части задания можно скачать данные по отзывам на фильмы с сайта IMDB (https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews), в которых для каждого отзыва поставлена семантическая оценка - "позитивный" или "негативный". 

In [ ]:
# Parsing

In [122]:
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек 
import time          # Библиотека для времени
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

In [118]:
def getPageLinks(month_number):
    page_link = 'https://nplus1.ru/news/2021/' + month_number
    response = requests.get(page_link)
    if not response.ok:
        return [] 

    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    news_links = soup.findAll(lambda tag: tag.name == 'a' and '/news/2021/' in tag.get('href'))
    news_links = ['https://nplus1.ru' + link.attrs['href'] for link in news_links]

    return news_links

In [121]:
month_numbers=['01','02','03','04','05','06','07','08','09','10','11','12']
month_number=month_numbers[2]
newwz_links = getPageLinks(month_number)
print(newwz_links[:2],'\n',len(newwz_links))

['https://nplus1.ru/news/2021/03/31/humanized-muscle-pig', 'https://nplus1.ru/news/2021/03/31/JCVI-syn30126'] 
 200


In [197]:
news_page = newwz_links[7]
response = requests.get(news_page, headers={'User-Agent': UserAgent().chrome})

html = response.content
soup = BeautifulSoup(html,'html.parser')

Посмотрим, как можно вытащить статистику просмотров, комментариев, а также числа загруженных видео и фото, связанных с нашим мемов. Всё это добро хранится справа вверху под тэгами `"dd"` и с классами  `"views"`, `"videos"`, `"photos"` и `"comments"`

In [167]:
difficulty = soup.find(lambda tag: tag.name == 'a' and 'difficult' in tag.get('href'))
difficulty_coef=float(difficulty.get("href").split(sep='/')[-1])
print(difficulty.text,'\n\n',difficulty_coef)


Сложность
3.3
 

 3.3


In [187]:
def getDifficulty(soup):
    try:
        difficulty = soup.find(lambda tag: tag.name == 'a' and 'difficult' in tag.get('href'))
        difficulty_coef=float(difficulty.get("href").split(sep='/')[-1])
    except:
        difficulty_coef=None
    return difficulty_coef

In [173]:
rubric = soup.find(lambda tag: tag.name == 'a' and 'rubric' in tag.get('href'))
rubric=(rubric.text)
print(rubric)

Астрономия


In [188]:
def getRubric(soup):
    try:
        rubric = soup.find(lambda tag: tag.name == 'a' and 'rubric' in tag.get('href'))
        rubric=(rubric.text)
    except:
        rubric=None
    return rubric

In [186]:
import datetime
time = soup.find(lambda tag: tag.name == 'time' and 'datePublished' in tag.get('itemprop'))
#rubric=(rubric.text)
print(time.text.split(sep='\n')[1])
print(datetime.datetime.strptime(time.text.split(sep='\n')[1], '%H:%M').time())
seconds_count=(datetime.datetime.strptime(time.text.split(sep='\n')[1], '%H:%M')-datetime.datetime.strptime('00:00', '%H:%M')).total_seconds()
print(seconds_count)

21:13
21:13:00
76380.0


In [189]:
def getTime(soup):
    try:
        time = soup.find(lambda tag: tag.name == 'time' and 'datePublished' in tag.get('itemprop'))
        seconds_count=(datetime.datetime.strptime(time.text.split(sep='\n')[1], '%H:%M')-datetime.datetime.strptime('00:00', '%H:%M')).total_seconds()
    except:
        seconds_count=None
    return seconds_count

Снова запихнём всё это в небольшую функцию.

In [47]:
def getStats(soup, stats):
    try:
        obj = soup.find('div', attrs={'class':stats})
        obj = obj.get("title")
        obj = obj.split()[0]
        obj = int(obj.replace(',', ''))
    except:
        obj=None
    return obj

Всё готово! 

In [193]:
views = getStats(soup, stats='views')
videos = getStats(soup, stats='videos')
photos = getStats(soup, stats='photos')
comments = getStats(soup, stats='comments')

print("Просмотры: {}\nВидео: {}\nФото: {}\nКомментарии: {}".format(views, videos, photos, comments))

NameError: name 'getStats' is not defined

In [198]:
seconds = getTime(soup)
rubric = getRubric(soup)
difficulty = getDifficulty(soup)
print("seconds: {}\nrubric: {}\ndifficulty: {}".format(seconds, rubric, difficulty))

seconds: 53520.0
rubric: Астрономия
difficulty: 3.1


Еще из интересного и исследовательского —  достанем дату и время добавления мема. Если посмотреть на страницу в браузере, можно подумать, что максимум информации, который мы можем вытащить - это число лет, прошедших с момента публикации —  `Added 4 years ago by NovaXP`. Однако мы так просто сдаваться не будем, полезем в кишки html и откопаем там кусок, ответственный за эту надпись:



Ага! Вот и подробности по дате добавления, с точностью до минуты. Элементарно

In [49]:
date = soup.findAll('abbr', attrs={'class':'timeago'})[1].attrs['title']
date

'2022-01-18T16:56:25-05:00'

На самом деле, парсеры — дело непредсказуемое. Часто страницы, которые мы парсим, имеют очень неоднородну структуру. Например, если мы парсим мемы, на части страниц может быть указано описание, а на части нет. Как только код впервые натыкается на отсутствие описания, он выдаёт ошибку и останавливается. Чтобы нормально собрать все данные, приходится прописывать исключения. Вроде бы, хранилище мемов хорошо оборудовано и никаких внештатных ситуаций происходить не должно. Тем не менее, очень не хочется проснуться утром и увидеть, что код сделал 20 итераций, нарвался на ошибку и отрубился.  Чтобы такого не произошло, можно, например, использовать конструкцию `try - except` и просто обрабатывать неугодные нам ошибки. Про исключения можно почитать [на просторах интернета](https://pythonworld.ru/tipy-dannyx-v-python/isklyucheniya-v-python-konstrukciya-try-except-dlya-obrabotki-isklyuchenij.html). В нашем же случае до ошибки можно и не доводить, а предварительно проверять, есть ли необходимый элемент на странице или нет при помощи обычного `if - else`, и уже после этого пытаться его распарсить.

Например, мы хотим вытащить статус мема, для этого найдем окружающие его тэги:

In [50]:
properties = soup.findAll('div', attrs={'class':'detail'})
properties[0]

<div class="detail">
<span>Status:</span>
<div class="tooltip-popup">
<span>
confirmed
</span>
</div>
</div>

In [51]:
prop = properties[0].findAll("span")
prop = [p.text.strip() for p in prop]
prop = {prop[0][:-1]:prop[1]}
prop

{'Status': 'confirmed'}

In [52]:
def _clean_property(prop):
    prop = prop.findAll("span")
    prop = [p.text.strip() for p in prop]
    prop = {prop[0][:-1]:prop[1]}
    return prop

In [53]:
for prop in properties:
    print(_clean_property(prop))

{'Status': 'confirmed'}
{'Origin': 'Tumblr'}
{'Year': '2010'}
{'Type': 'Character,\nAnimal,\nSlang'}


## Truthy & Falsy python 
https://www.freecodecamp.org/news/truthy-and-falsy-values-in-python/

In [54]:
if None:
    print(1)
else:
    print(2)

2


In [55]:
if [1, 2, 3]:
    print(1)

1


In [56]:
if []:
    print(1)

Такой код позволяет обезопасить себя от ошибок в коде. В данном случае, мы можем переписать всю конструкцию с `if - else` в виде одной удобной строки. Эта строка проверит полон ли респонса `meme_status` и ежели нет, то выдаст пустоту.

In [57]:
"Hello" if True else "Bye"

'Hello'

In [58]:
1 if ("a" in ['a', 'b']) or ('c' in [1, 4]) else 0

1

По аналогии можно вытащить всю остальную информацию со страницы.

In [59]:
def getProperties(soup):
    """
        Возвращает список (tuple) с названием, статусом, типом, 
        годом и местом происхождения и тэгами
        
        soup: объект bs4.BeautifulSoup 
            суп текущей страницы
    
    """
    # название - идёт с самым большим заголовком h1, легко найти
    meme_name = soup.find('section', attrs={'class':'info'}).find('h1').text.strip()
    
    clean_properties = {'Name':meme_name}
    properties = soup.findAll('div', attrs={'class':'detail'})
    for prop in properties:
        clean_properties.update(_clean_property(prop))
        
    return clean_properties

In [60]:
getProperties(soup)

{'Name': 'Doge Meme',
 'Origin': 'Tumblr',
 'Status': 'confirmed',
 'Type': 'Character,\nAnimal,\nSlang',
 'Year': '2010'}

Свойства мема собрали. Теперь собираем по аналогии его текстовое описание. 

In [61]:
def getText(soup):
    """
        Возвращает текстовые описания мема
        
        soup: объект bs4.BeautifulSoup 
            суп текущей страницы
            
    """
    
    # достаём все тексты под картинкой
    body = soup.find('section', attrs={'class':'bodycopy'})
    
    # раздел about (если он есть), должен идти первым, берем его без уточнения класса
    meme_about = body.find('p')
    meme_about = "" if not meme_about else meme_about.text
    
    # раздел origin можно найти после заголовка Origin или History, 
    # находим заголовок, определяем родителя и ищем следущего ребенка - наш раздел
    meme_origin = body.find(text='Origin') or body.find(text='History')
    meme_origin = "" if not meme_origin else meme_origin.parent.find_next().text
    
    # весь остальной текст (если он есть) можно запихнуть в одно текстовое поле
    if body.text:
        other_text = body.text.strip().split('\n')[4:]
        other_text = " ".join(other_text).strip()
    else:
        other_text = ""
        
    return {'About':meme_about, 'Origin':meme_origin, 'Other_text':other_text}

In [62]:
text_fields = getText(soup)

print("О чем мем:\n{}\n\nПроисхождение:\n{}\n\nОстальной текст:\n{}...\n"\
      .format(text_fields['About'], text_fields['Origin'], text_fields['Other_text'][:200]))

О чем мем:
Doge (pronounced /ˈdoʊdʒ/ DOHJ) is a slang term for "dog" that is primarily associated with pictures of Shiba Inus (nicknamed "Shibe") and internal monologue captions on Tumblr. These photos may be photoshopped to change the dog's face or captioned with interior monologues in Comic Sans font. Starting in 2017, Ironic Doge formats gained prevalence over the original wholesome version.

Происхождение:
The use of the misspelled word "doge" to refer to a dog dates back to June 24th, 2005, when it was mentioned in an episode of Homestar Runner's puppet show. In the episode titled "Biz Cas Fri 1"[2], Homestar calls Strong Bad his "d-o-g-e" while trying to distract him from his work.

Остальной текст:
OriginThe use of the misspelled word "doge" to refer to a dog dates back to June 24th, 2005, when it was mentioned in an episode of Homestar Runner's puppet show. In the episode titled "Biz Cas Fri 1"...



Наконец, создадим функцию, возвращающую всю информацию по текущему мему

In [63]:
def getMemeData(meme_page):
    """
        Запрашивает данные по странице, возвращает обработанный словарь с данными
        
        meme_page: string
            ссылка на страницу с мемом
    
    """
    
    # запрашиваем данные по ссылке
    response = requests.get(meme_page, headers={'User-Agent': UserAgent().chrome})
    
    if not response.ok:
        # если сервер нам отказал, вернем статус ошибки 
        return response.status_code
    
    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')

    # используя ранее написанные функции парсим информацию
    views = getStats(soup=soup, stats='views')
    videos = getStats(soup=soup, stats='videos')
    photos = getStats(soup=soup, stats='photos')
    comments = getStats(soup=soup, stats='comments')

    # дата
    date = soup.findAll('abbr', attrs={'class':'timeago'})[1].attrs['title']

    # имя, статус, и т.д.
    properties = getProperties(soup=soup)

    # текстовые поля
    text_fileds = getText(soup=soup)

    # составляем словарь, в котором будут хранится все полученные и обработанные данные
    data_row = {"date_added":date, "views":views, 
                "videos":videos, "photos":photos, "comments":comments}
    data_row.update(properties)
    data_row.update(text_fileds)

    return data_row

In [64]:
data_row = getMemeData('http://knowyourmeme.com/memes/doge')

А теперь подготовим табличку, чтобы в неё записывать всё ~~награбленные~~ честно полученные данные, добавим в неё первую полученную строку и полюбуемся на результат

In [65]:
final_df = pd.DataFrame(columns=['Name', 'Status', 'Type', 'Year', 'Origin',
                                 'date_added', 'views', 'videos', 'photos', 'comments', 
                                 'About', 'Origin', 'Other_text'])

In [66]:
final_df = final_df.append(data_row, ignore_index=True)

In [67]:
final_df

,Name,Status,Type,Year,Origin,date_added,views,videos,photos,comments,About,Origin,Other_text
0,Doge Meme,confirmed,"Character,\nAnimal,\nSlang",2010,"The use of the misspelled word ""doge"" to refer...",2022-01-18T16:56:25-05:00,13842276,103,1776,925,Doge (pronounced /ˈdoʊdʒ/ DOHJ) is a slang ter...,"The use of the misspelled word ""doge"" to refer...","OriginThe use of the misspelled word ""doge"" to..."


Первый мем оказался в наших рукак. Еще раз убедимся что всё работает — пройдемся по списку из ссылок на мемы, полученных ранее в перменной `meme_links`.

In [68]:
from tqdm import tqdm_notebook

In [69]:
meme_links

['http://knowyourmeme.com/memes/us-or-ps5',
 'http://knowyourmeme.com/memes/events/when-we-were-young-festival',
 'http://knowyourmeme.com/memes/kirby-fishing',
 'http://knowyourmeme.com/memes/mario-karted',
 'http://knowyourmeme.com/memes/goots',
 'http://knowyourmeme.com/memes/material-gworl',
 'http://knowyourmeme.com/memes/emotional-damage',
 'http://knowyourmeme.com/memes/trad-boywife-trad-husband',
 'http://knowyourmeme.com/memes/look-pim-i-know-its-our-job',
 'http://knowyourmeme.com/memes/20-fingers-gillette-short-dick-man',
 'http://knowyourmeme.com/memes/nyquil-chicken',
 'http://knowyourmeme.com/memes/events/ninja-vs-pokimane',
 'http://knowyourmeme.com/memes/events/microsofts-acquisition-of-activision-blizzard',
 'http://knowyourmeme.com/memes/subcultures/animusic',
 'http://knowyourmeme.com/memes/lost-in-the-fire-dance-challenge',
 'http://knowyourmeme.com/memes/nikocado-avocados-ass-pic']

In [70]:
for meme_link in tqdm_notebook(meme_links):
    try: 
        data_row = getMemeData(meme_link)
        final_df = final_df.append(data_row, ignore_index=True)
        time.sleep(0.3)
    except:
        continue

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/16 [00:00<?, ?it/s]

In [71]:
final_df.drop_duplicates(inplace=True)

In [72]:
final_df.shape

(17, 13)

In [73]:
final_df.head()

,Name,Status,Type,Year,Origin,date_added,views,videos,photos,comments,About,Origin,Other_text
0,Doge Meme,confirmed,"Character,\nAnimal,\nSlang",2010,"The use of the misspelled word ""doge"" to refer...",2022-01-18T16:56:25-05:00,13842276,103,1776,925,Doge (pronounced /ˈdoʊdʒ/ DOHJ) is a slang ter...,"The use of the misspelled word ""doge"" to refer...","OriginThe use of the misspelled word ""doge"" to..."
1,Us or PS5? Meme,submission,"Participatory Media,\nViral Video",2021,"In November 2021, TikTok[1][2] dance and conte...",2022-01-20T20:37:57-05:00,2230,3,0,24,Us or PS5? or Them or PS5? refers to a viral v...,"In November 2021, TikTok[1][2] dance and conte...","OriginIn November 2021, TikTok[1][2] dance and..."
2,When We Were Young Festival Event,submission,"Convention,\nPerformance,\nPromotion",2022,,2022-01-21T04:59:41-05:00,3106,0,29,12,"The When We Were Young Festival, alternatively...",,"BackgroundOn January 18th, 2022, the official ..."
3,Kirby Fishing Meme,submission,"Exploitable,\nFan Art",2022,The trailer to Kirby and the Forgotten Land wa...,2022-01-20T19:33:15-05:00,253,3,11,0,Kirby Fishing is a scene from the trailer to K...,The trailer to Kirby and the Forgotten Land wa...,SpreadThe moment quickly inspired jokes about ...
4,Mario Karted Meme,submission,Viral Video,2014,"The original ""Get Mario Karted"" video was uplo...",2022-01-19T02:16:42-05:00,127,9,0,0,Mario Karted is a video game fail trend in whi...,"The original ""Get Mario Karted"" video was uplo...","OriginThe original ""Get Mario Karted"" video wa..."


Отлично! Всё работает, мемы качаются, данные наполняются и всё было бы хорошо, если бы не одно но — количество запросов, которое нам придётся сделать, чтобы всё получить.

# 2.  Прячемся от стражников

## 2.1 Когда работающий код больше не работает

Вот он! Тот самый момент абсолютного триумфа, когда код дописан и всё, что нам, мирным собирателям, остаётся — запустить наш код на одну ночку. Кажется, что через страсть мы преобрели силу. Запускаем наш код по всем  страницам с мемами. На всякий случай обернём наш цикл в `try-except`. Мало ли что там с этими мемами бывает. 

In [ ]:
# Немного красивых циклов. При желании пакет можно отключить и 
# удалить команду tqdm_notebook из всех циклов
from tqdm import tqdm_notebook


final_df = pd.DataFrame(columns=['name', 'status', 'type', 'origin_year', 'origin_place',
                                 'date_added', 'views', 'videos', 'photos', 'comments', 
                                 'tags', 'about', 'origin', 'other_text'])

for page_number in tqdm_notebook(range(1500), desc='Pages'):
    # собрали хрефы с текущей страницы
    meme_links = getPageLinks(page_number)  
    for meme_link in tqdm_notebook(meme_links, desc='Memes', leave=False):
        # иногда с первого раза страничка не парсится
        for i in range(3):
            try:
                # пытаемся собрать по мему немного даты
                data_row = getMemeData(meme_link)           
                # и закидываем её в таблицу
                final_df = final_df.append(data_row, ignore_index=True)  
                # если всё получилось - выходим из внутреннего цикла
                break
            except:
                # Иначе, пробуем еще несколько раз, пока не закончатся попытки
                print('AHTUNG! parsing once again:', meme_link)
                continue

C:\Users\mashk\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


Pages:   0%|          | 0/1783 [00:00<?, ?it/s]

C:\Users\mashk\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


Memes:   0%|          | 0/16 [00:00<?, ?it/s]

Memes:   0%|          | 0/16 [00:00<?, ?it/s]

Memes:   0%|          | 0/16 [00:00<?, ?it/s]

Memes:   0%|          | 0/16 [00:00<?, ?it/s]

Memes:   0%|          | 0/16 [00:00<?, ?it/s]

AHTUNG! parsing once again: http://knowyourmeme.com/memes/good-soldiers-follow-orders
AHTUNG! parsing once again: http://knowyourmeme.com/memes/events/drake-bell-child-endangerment
AHTUNG! parsing once again: http://knowyourmeme.com/memes/explain-in-familiar-terms
AHTUNG! parsing once again: http://knowyourmeme.com/memes/yes-rico-kaboom
AHTUNG! parsing once again: http://knowyourmeme.com/memes/yes-rico-kaboom
AHTUNG! parsing once again: http://knowyourmeme.com/memes/yes-rico-kaboom


Сон был прекрасным! Солнце только-только взошло из-за горизонта, мы уже бежим за компьютер смотреть мемы и видим, что огромное число мемов не скачалось.


Конечно же, вполне естественной реакцией будет нажать на первую же ссылку, перейти в мемохранилище и увидеть, что нас забанили.


Все наши реквесты остались без респонсов. 

## 2.2 Тор - сын Одина

Иногда серверу надоедает общаться с одним и тем же человеком, делающим кучу запросов и сервер банит его. К сожалению, не только у людей запас терпения ограничен.

Приходится маскироваться. Для такой маскировки можно использовать разные способы, более того, один из них мы уже использовали, когда притворились человеком в нашем `request-header`. Для текущей же задачи, когда нас вероломно заблокировали по IP, нужно искать способы помощнее, чтобы иметь возможность этот IP менять. Конечно, как вариант можно было бы использовать прокси-сервера, тогда мы бы имели в запасе некоторое количество разных IP адресов, которые можно подставлять по мере "забанивания". Однако в этом подходе есть пара проблем: первая - нужно где-то раздобыть эти прокси, вторая - а что если ограниченного числа адресов нам не хватит и нужно больше?

В таком случае лучше всего нам подойдёт [Tor](https://www.torproject.org/). Вопреки пропагандируемому мнению, Tor используется не только преступниками, педофилами и прочими нехорошими террористами. Это, мягко говоря, далеко не так, и мы, мирные собиратели данных, являемся тому подтверждением. Всем прелестям, связанным с работой Tor, можно было бы посвятить несколько больших статеек, что собственно говоря уже и сделано. Подробнее про это можно почитатать по следующим ссылкам: 

* [Как работает Tor](https://geektimes.ru/post/277578/)
* [Методы анонимности в сети](https://habrahabr.ru/post/204266/)
* [Прокси-сервер с помощью Tor](https://habrahabr.ru/company/etagi/blog/315002/)

Мы же ограничимся только функциональной частью, а именно без углубления в детали опишем шаги, которые нужно предпринять для того, чтобы использовать возможности Tor для обхода блокировки. Для начала полюбуемся на свой ip-адрес. Для этого сделаем get-запрос к сайту, который возвращяет наш IP

In [ ]:
def checkIP():
    ip = requests.get('http://checkip.dyndns.org').content
    soup = BeautifulSoup(ip, 'html.parser')
    print(soup.find('body').text)

In [ ]:
checkIP()

Current IP Address: 85.174.235.145


Заменить свой ip через Tor можно двумя путями. Простой - через браузер, а сложный - через небольшие махинации с настройками.

Скачаем браузерный [tor,](https://www.torproject.org/download/download) чтобы лёгкий путь был совсем прост. Для сложного пути в довесок к браузеру поставим tor через консоль. 
- Linux - нам поможет команда `apt-get install tor`, 
- Mac - сделаем это [в рамках brew](https://www.torproject.org/docs/tor-doc-osx.html.en), `brew install tor`. 
- Windows - ~~нам поможет установка другой операционной системы.~~ 

## 2.3 Путь первый 

Теперь запускаем свежескачанный браузер и оставляем его открытым.  Менять ip нам поможет библиотека `PySocks`. Конечно же, её нужно установить, скопировав в терминал `pip3 install PySocks`.  

Браузер тора по умолчанию использует порт номер 9150. В питоне при помощи библиотек socks и socket можно задать дефолтный порт для подключения. В результате текущая сессия будет использовать именно этот порт при отправке любого запроса, а значит – запросы будут посылаться из-под запущенного тора.

In [ ]:
import socks
import socket
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
socket.socket = socks.socksocket

Посмотрим на свой новый ip-aдрес. 

In [ ]:
checkIP()

Попробуем обратиться к мемохранилищу с нового ip-адреса.

In [ ]:
data_row = getMemeData('http://knowyourmeme.com/memes/doge')

for key, value in data_row.items():
    print(key.capitalize()+":", str(value)[:200], end='\n\n')

Бан снят. Стражники мемов ничего не заподозрили и пустили нас в сокровищницу. Чашу нашего респонса снова переполняет контент. Через силу мы обрели мощь. 

При желании, можно выяснить одну занимательную вещь: при базовых настройках, Тор-браузер меняет ip каждые 10 минут. Но что делать, если сервер банит нас быстрее?  Всё просто, в папке, куда был установлен Tor найдём файлик с настройками под названием torrc (на маке он лежит по адресу `~/Library/Application Support/TorBrowser-Data/torrc`, если не получится найти - добро пожаловать [сюда](https://tor.stackexchange.com/questions/11866/cant-find-torrc-file-on-mac)) и отредактируем его. Добавим строки: 

```
CircuitBuildTimeout 10
LearnCircuitBuildTimeout 0
MaxCircuitDirtiness 10
```

Минимально возможный период для обновления ip составляет 10 секунд. Установим туда эту цифру и попробуем поиграться. 

In [ ]:
for i in range(10):
    checkIP()
    time.sleep(5)

Current IP Address: 199.249.230.177
Current IP Address: 199.249.230.177
Current IP Address: 51.158.171.35
Current IP Address: 51.158.171.35
Current IP Address: 51.81.87.40
Current IP Address: 51.81.87.40
Current IP Address: 185.220.100.245
Current IP Address: 185.220.100.245
Current IP Address: 193.218.118.167
Current IP Address: 193.218.118.167


Действительно, смена ip происходит примерно раз в 10 секунд. Для наших целей по скачке мемов было достаточно и базовых настроек. Бан наступал примерно через 20 минут после начала работы кода. 

1. Открываем браузер;
2. Запускаем кусок кода с подгрузкой библиотек;
3. Запускаем цикл по мемам 
4. .....
5. Profit

In [ ]:
final_df = pd.DataFrame(columns=['name', 'status', 'type', 'origin_year', 'origin_place',
                                 'date_added', 'views', 'videos', 'photos', 'comments', 
                                 'tags', 'about', 'origin', 'other_text'])


for page_number in tqdm_notebook(range(1455), desc='Pages'):
    # собрали хрефы с текущей страницы
    meme_links = getPageLinks(page_number)  
    for meme_link in tqdm_notebook(meme_links, desc='Memes', leave=False):
        # иногда с первого раза страничка не парсится
        for i in range(5):
            try:
                # пытаемся собрать по мему немного даты
                data_row = getMemeData(meme_link)           
                # и закидываем её в таблицу
                final_df = final_df.append(data_row, ignore_index=True)  
                # если всё получилось - выходим из внутреннего цикла
                break
            except:
                # Иначе, пробуем еще несколько раз, пока не закончатся попытки
                continue
                
    final_df.to_csv('MEMES_{}.csv'.format(page_number))
    if page_number//10:
        print(final_df.shape)
final_df.to_csv('MEMES_full.csv')

Все мемы в наших руках. Можно приступать к варке фичей и моделированию. 

## 2.4 Путь второй - Stem

https://stem.torproject.org/

Stem is a Python controller library for Tor. With it you can use Tor's control protocol to script against the Tor process, or build things such as Nyx. Stem's latest version is 1.8 (released December 29th, 2019).

Позволяет подключаться к портам тора и более гибко настраивать взаимодействие с ним из питона

Теперь весь код для париснга можно перевести на рельсы этого небольшого скрипта, заменив все реквесты на торовские и менять IP адрес через каждые несколько реквестов к серверу, на который внезапно обрушится огромное число запросов из всех уголков света.

Напоследок, хотелось бы сказать пару слов о парсинге вообще и при помощи Тора в частности. Добывать себе данные - это стильно, модно и в принципе интересно, можно получить датасеты, которых еще никто никогда не обрабатывал, сделать что-то новое, посмотреть, наконец, на все мемы мира сразу. Однако не стоит забывать, что ограничения, введенные сервером, в том числе баны, появились не просто так, а в целях защиты сайта от недоброжелательных ковровых бомбардировок запросами и DDoS-атак. К чужому труду стоит относится с уважением, и даже если у сервера никакой защиты нет, - это еще не повод неограниченно забрасывать его реквестами, особенно если это может привести к его отключению - [уголовное наказание](http://sd-company.su/article/security/ddosataka-ugolovnaya-otvetstvennost) никто не отменял. Успешных и безопасных вам исследований!


## Почиташки 

* [Годная книга](https://github.com/FUlyankin/Parsers/blob/master/Ryan_Mitchell_Web_Scraping_with_Python-_Collecting_Data_from_the_Modern_Web_2015.pdf) про парсинг на английском языке. 
* [Неплохая инструкция](https://jarroba.com/anonymous-scraping-by-tor-network/) о самостоятельном парсинге через Tor без использования чужих готовых классов. 
* [Димин репозиторий](https://github.com/DmitrySerg/memology) с исследованием мемов.
* [Оригинальный кодекс](http://starwars.wikia.com/wiki/Code_of_the_Sith) адепта тёмной стороны силы.